In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}


In [2]:
# 다음에서 상영영화 이름 리스트

data = requests.get('https://search.daum.net/search?w=tot&q=%ED%98%84%EC%9E%AC%EC%83%81%EC%98%81%EC%9E%91&rtmaxcoll=0SP&DA=MOR', headers=headers)
html_data = BeautifulSoup(data.text, 'html.parser')

# name -> 네이버에서 사용
name_list = []

data_list = html_data.find_all('a', class_='fn_tit txt_ellip')
num = len(data_list)

for i in range(num):
    name_list.append(''.join(data_list[i].text.split()))
    

# url -> 다음 리뷰 긁어올때 사용
url_list = []
url = 'https://search.daum.net/search'

data_list = html_data.find_all('div', class_='wrap_thumb')
data_list = data_list[:num]
for i in range(num):
    movie_url = url + data_list[i].find('a')['href']
    url_list.append(movie_url)
    
    
import re
# reivew 링크 저장
movie_code_list = []
for url in url_list:
    data = requests.get(url)
    daum_filter = BeautifulSoup(data.text, 'html.parser')
    daum_filter = daum_filter.find_all(attrs={'disp-attr': 'EM1'})[0]
    try:
        link = daum_filter.find_all('dd')[4].find('c-link').get('data-href')
        num = re.search(r'\d+', link).group()
        movie_code_list.append(int(num))
    except:
        movie_code_list.append(0)
        
print(len(name_list), len(url_list), len(movie_code_list))

20 20 20


In [27]:

url = 'https://gateway.kinolights.com/graphql'
num = 10
review_list = []

for i in movie_code_list:
    temp = []
    if i != 0:
        for j in range(0, 210, 10):
            payload = {
                'operationName' : 'QueryReviews',
                'query' : "fragment FragmentReviewField on UserReviewOutput {\n  id\n  reviewTitle\n  review\n  reviewLink\n  isPrivate\n  isPreview\n  hasSpoiler\n  likeCount\n  commentCount\n  sharedCount\n  viewCount\n  deletedUserId\n  createdAt\n  isMyLike\n  userIndexRating {\n    index\n    __typename\n  }\n  userStarRating {\n    star\n    __typename\n  }\n  movie {\n    id\n    titleKr\n    posterImage {\n      path\n      __typename\n    }\n    __typename\n  }\n  reviewer {\n    id\n    name\n    isPremium\n    profileImage {\n      path\n      __typename\n    }\n    __typename\n  }\n  watchedAt {\n    watchedAt\n    __typename\n  }\n  blindStatus\n  isReportedByMe\n  __typename\n}\n\nquery QueryReviews($offset: Int = 0, $limit: Int = 10, $movieId: Int!, $orderBy: ReviewMoviesOrderType!, $orderOption: OrderOptionType!) {\n  reviews(\n    offset: $offset\n    limit: $limit\n    movieId: $movieId\n    orderBy: $orderBy\n    orderOption: $orderOption\n  ) {\n    ...FragmentReviewField\n    __typename\n  }\n}\n",
                'variables' : {'offset': j, 'limit': num, 'movieId': i, 'orderBy': "LIKE", 'orderOption': "DESC"}
            }
            
            try:
                response =requests.post(url, json=payload, headers=headers)
                data_list = json.loads(response.text)['data']['reviews']
                for k in range(num):
                    review = data_list[k]['reviewTitle']
                    temp.append(review)
            except:
                break
        review_list.append(temp)
    else:  
        review_list.append([0])

In [28]:
num = len(review_list)

for i in range(num):
    try:
        print(i, ':', review_list[i][:2])
    except:
        print('hum : ', i, ':', review_list[i][:2])

0 : ['실화 바탕으로 여성 주연들이 활약하는 여성감독 영화라는 점에서 의미는 있다. 그러나 캐릭터들을 이토록 작위적으로 구성할 수가 있나?', '실화를 모티브로 하였지만\n실화는 아닌 이야기.\n실화를 그대로 다뤘다면\n오히려 욕먹었을 이야기를\n적절하게 영화화하였다.\n무엇보다 K-아줌마와 같은\n끔찍한 혼종은 없어서 좋았다.']
1 : ['나는 아직도 1부가 흥행하지 못한 이유를 모르겠다', '기대하고 봐도 재미있고 기대없이 봐도 재미있는 영화!!!!']
2 : ['나는 (누군가의) 별이다', '다양성을 이야기하며 다양성을 잃어가고 있다.']
3 : ['한 발짝씩 내디딜 때마다 고통을 향해 나아가는 한 남자의 인생이 신(GOD)이 아닌 개(DOG)로부터 구원받는다.', '아무리 개들에게 사랑을 받아도, 사람의 사랑도 필요하다.']
4 : ['긍정으로 가득찬···조금의 고난과 갈등을 보여주며···재미를 선사하는 애니였다😄😄 몇년만에 이렇게 편안하게 극장관람을 했는지 모르겠다', '이런 영화야 말로 가족들과 편안하게 볼 수 있는 영화다\n게다가 4DX 포맷으로 본다면 비행 장면이 많아서 더 극적인 감각을 느낄 수 있어서 더 재밌다']
5 : ['다시는 반복되지 않아야 할 역사.', '결말을 알고봐도 몰입감이 장난아님..\n보고나면 짜증이 날지경..\n연기가 살벌하다']
6 : ['장엄한 북소리.\n감히 어떤 말로도 표현할 수 없을 만큼 가슴이 벅차올랐다.', '불필요한 죽음을 부정할 필요도 있다.']
7 : ['아쉬움이 남는 이별이 나을 때가 있다. 드라마를 안 본 사람들에게는 불친절했고, 본 사람들을 모욕한 영화', '드라마 팬들을 위한 영화 같다 대만 최애 드라마지만 영화는 아쉬움.. ']
8 : [0]
9 : ['신선하고 웃겼으나 시대가 맞지않아 공감이 어려웠다.', '빛나는 캐릭터들로 써 내려간 사랑과 구원']
10 : ['고레에다 히로카즈의 <괴물> 속 우주는 계속 팽창하며, 마침내 모든 것이 무너지면 시간은 되돌아간다. 그렇게 우리의 관점도 역전

In [30]:
df = pd.DataFrame()
df['name'] = name_list
df['code'] = movie_code_list
df['review'] = review_list
df.to_csv('daum.csv')